<a href="https://colab.research.google.com/github/Aditya0721/Recommender/blob/master/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **# The main idea behind the generation of the Recommendation system is to recommend posts using both Collaborative and Content Based Filtering Methods. For Collaborative based filtering we will find out all the categories and find the cosine similarity using user-user collaborative filtering.**
```



# **DATA PREPROCESSING**

## Uploading the datasets and storing it in dataframes

In [0]:
import pandas as pd
from google.colab import files
import io

In [0]:
uploaded = files.upload()

Saving posts.csv to posts.csv
Saving users.csv to users.csv
Saving views.csv to views.csv


In [0]:
users_df = pd.read_csv('users.csv')
users_df.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [0]:
posts_df = pd.read_csv('posts.csv')
posts_df.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [0]:
views_df = pd.read_csv('views.csv')
views_df.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


## DETAILS OF THE DATAFRAMES

In [0]:
users_df.describe()

,_id,name,gender,academics
count,118,118,118,118
unique,118,118,3,3
top,5e7b30a6cfc8b713f5ac7d95,Harishankar Dubey,male,undergraduate
freq,1,1,72,68


In [0]:
views_df.describe()

,user_id,post_id,timestamp
count,1449,1449,1449
unique,118,495,1449
top,5d60098a653a331687083238,5ec1fd0974f7660d73aa0fd5,2020-04-06T17:50:28.504Z
freq,230,18,1


In [0]:
posts_df.describe()

,_id,title,category,post_type
count,493,493,465,493
unique,493,477,231,4
top,5eb1407c10426255a7aa9fdb,PENCIL RENDERING,Photography,artwork
freq,1,3,81,241


**checking for null values**

In [0]:
users_df.isnull().any()


_id          False
name         False
gender       False
academics    False
dtype: bool

In [0]:
views_df.isnull().any()

user_id      False
post_id      False
timestamp    False
dtype: bool

In [0]:
posts_df.isnull().any()

_id           False
title         False
category       True
 post_type    False
dtype: bool

**we have got null values in the category column of the posts dataset**

**as its a categorical variable we have to fill the null values considering the title and post_type**

**counting total number of null values in posts_df**

In [0]:
posts_df['category'].isna().sum()

28

**finding title and post_type for all 28 null values**

In [0]:
posts_df[posts_df['category'].isna()]

,_id,title,category,post_type
24,5ddeb6e80eb5e25a8a07f065,Library Managment System: Software Requirement...,NaN,project
25,5de179d80eb5e25a8a07f079,Navigation system using BFS DFS algorithms,NaN,project
29,5dee9b5042a8854bf6eabaaf,Computer Aided Machine Drawing (CAMD),NaN,project
65,5e3ea110eab55d319938a7a7,OS,NaN,project
88,5e4c3873f5561b1994c8e3d9,Bill Calculation(PHP),NaN,project
93,5e4da502f5561b1994c8e42c,Prime No.(PHP),NaN,project
94,5e4da7c8f5561b1994c8e439,Factorial of a No.(PHP),NaN,project
95,5e4dab15f5561b1994c8e446,Palindrome No.(PHP),NaN,project
96,5e4ed85af5561b1994c8e470,Factorial (.net),NaN,project
97,5e4ed8ccf5561b1994c8e47d,Leap year checking (.net),NaN,project


**As we can see most of the null values are for the post_types project we will create a new category called project and assign it to the null values and when ever a user leaves the category field empty we will assign the category field same as the post_type as default**

In [0]:
posts_df[posts_df['category'].isna()]='project'

**As we have imputed the null values lets check again**

In [0]:
posts_df.isna().any()

_id           False
title         False
category      False
 post_type    False
dtype: bool

**Now lets see how many unique categories are present in the dataset**

In [0]:
posts_df['category'].unique()

array(['Plant Biotechnology',
       'Artificial Intelligence|Machine Learning|Information Technology',
       'Operating Systems', 'Drawings', 'Competition Laws', 'Eco System',
       'Economic Policies', 'Graphic|Graphic Design', 'Painting',
       'Pen and ink', 'Computer Technology|Information Technology',
       'Drawings|Painting',
       'Graphic Design|Visual Arts|Illustration|Graphic',
       'Drawings|Calligraphy', 'Photography', 'Empowerment', 'project',
       'Video editing', 'Inorganic Chemistry', 'Programming languages',
       'Conceptual|Graphic Design', 'HR Management',
       'Human Resources|HR Management',
       'Mass Media|International Relations', 'Sculptures|Artistic design',
       'Fashion Design|Ceramics|Artistic design', 'Craft|Artistic design',
       'Fashion Design|Visual Arts|Conceptual|Artistic design',
       'Photography|Fashion Design|Visual Arts|Graphic Design|Artistic design',
       'Fashion Design|Visual Arts|Graphic Design|Artistic design|Graph

**As we can see some of the categories are combination of multiple categoris we will add more rows to the posts_df by splitting the categories and making one category for each row**

In [0]:
categories = {}
for i in posts_df['category']:
  categories.update({i:[]})
  for j in list(i.split("|")):
      categories[i].append(j) 
print(categories)

{'Plant Biotechnology': ['Plant Biotechnology'], 'Artificial Intelligence|Machine Learning|Information Technology': ['Artificial Intelligence', 'Machine Learning', 'Information Technology'], 'Operating Systems': ['Operating Systems'], 'Drawings': ['Drawings'], 'Competition Laws': ['Competition Laws'], 'Eco System': ['Eco System'], 'Economic Policies': ['Economic Policies'], 'Graphic|Graphic Design': ['Graphic', 'Graphic Design'], 'Painting': ['Painting'], 'Pen and ink': ['Pen and ink'], 'Computer Technology|Information Technology': ['Computer Technology', 'Information Technology'], 'Drawings|Painting': ['Drawings', 'Painting'], 'Graphic Design|Visual Arts|Illustration|Graphic': ['Graphic Design', 'Visual Arts', 'Illustration', 'Graphic'], 'Drawings|Calligraphy': ['Drawings', 'Calligraphy'], 'Photography': ['Photography'], 'Empowerment': ['Empowerment'], 'project': ['project'], 'Video editing': ['Video editing'], 'Inorganic Chemistry': ['Inorganic Chemistry'], 'Programming languages': [

**Now lets update posts_df**

In [0]:
updated_data =  []
for i in categories:
  dummy = posts_df[posts_df['category']==i]
  id = dummy['_id'].values[0]
  title = dummy['title'].values[0]
  post_type = dummy[' post_type'].values[0]
  for j in categories[i]:
      dict1 = {}
      dict1.update({'_id':id})
      dict1.update({'title':title})
      dict1.update({'category':j})
      dict1.update({' post_type':post_type})
      updated_data.append(dict1)

In [0]:
posts_df_updated = pd.DataFrame(updated_data)


**now lets check updated posts_df**

In [0]:
posts_df_updated.loc[:, ['_id', 'category', ' post_type']].head()

,_id,category,post_type
0,5d62abaa65218653a132c956,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Artificial Intelligence,blog
2,5d6d39567fa40e1417a4931c,Machine Learning,blog
3,5d6d39567fa40e1417a4931c,Information Technology,blog
4,5d7d23315720533e15c3b1ee,Operating Systems,blog


In [0]:
posts_df.loc[:, ['_id', 'category', ' post_type']].head()

,_id,category,post_type
0,5d62abaa65218653a132c956,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Drawings,artwork
4,5d80dfbc6c53455f896e600e,Competition Laws,blog


In [0]:
posts_df_updated.describe()

,_id,title,category,post_type
count,592,592,592,592
unique,232,228,235,4
top,5ec7abfdec493f4a26558860,MOSAIC ART,Computer Technology,blog
freq,7,10,24,374


**As we can see we have successfully created our posts dataframe now we will merge the dataframes**

##MERGING OF THE DATAFRAMES 

**We will merge views_df and posts_df_updated dataframes as they conatin all the columns and data that we will need for our recommendation system**

In [0]:
views_df.columns

Index(['user_id', 'post_id', 'timestamp'], dtype='object')

In [0]:
posts_df_updated.columns

Index(['_id', 'title', 'category', ' post_type'], dtype='object')

**As we know for merging we need a common column and post_id is the column in the both dataframes but we have to rename one of them to perform the merge operation**
**We will change rename _id to post id in the posts_df_updated column** 

In [0]:
posts_df_updated.rename(columns={'_id':'post_id'}, inplace=True)

In [0]:
posts_df_updated.columns

Index(['post_id', 'title', 'category', ' post_type'], dtype='object')

In [0]:
main_df = pd.merge(views_df, posts_df_updated)

In [0]:
main_df.columns

Index(['user_id', 'post_id', 'timestamp', 'title', 'category', ' post_type'], dtype='object')

In [0]:
main_df.head(10)

,user_id,post_id,timestamp,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Save Earth.,Visual Arts,artwork
1,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Save Earth.,Graphic Design,artwork
2,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Save Earth.,Artistic design,artwork
3,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Save Earth.,Graphic,artwork
4,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Save Earth.,Illustration,artwork
5,5ec3ba5374f7660d73aa1201,5ec821ddec493f4a2655889e,2020-05-24T10:49:55.177Z,Save Earth.,Visual Arts,artwork
6,5ec3ba5374f7660d73aa1201,5ec821ddec493f4a2655889e,2020-05-24T10:49:55.177Z,Save Earth.,Graphic Design,artwork
7,5ec3ba5374f7660d73aa1201,5ec821ddec493f4a2655889e,2020-05-24T10:49:55.177Z,Save Earth.,Artistic design,artwork
8,5ec3ba5374f7660d73aa1201,5ec821ddec493f4a2655889e,2020-05-24T10:49:55.177Z,Save Earth.,Graphic,artwork
9,5ec3ba5374f7660d73aa1201,5ec821ddec493f4a2655889e,2020-05-24T10:49:55.177Z,Save Earth.,Illustration,artwork


In [0]:
main_df.tail(20)

,user_id,post_id,timestamp,title,category,post_type
1730,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,2020-03-27T09:45:14.071Z,What sports will look like in the future,Computer Technology,blog
1731,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,2020-03-27T09:45:14.071Z,What sports will look like in the future,Robotics,blog
1732,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,2020-03-27T09:45:14.071Z,What sports will look like in the future,Data Science,blog
1733,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,2020-03-27T09:45:14.071Z,What sports will look like in the future,Information Technology,blog
1734,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,2020-03-27T09:45:14.071Z,What sports will look like in the future,Artificial Intelligence,blog
1735,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2020-03-26T21:39:16.764Z,What sports will look like in the future,Computer Technology,blog
1736,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2020-03-26T21:39:16.764Z,What sports will look like in the future,Robotics,blog
1737,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2020-03-26T21:39:16.764Z,What sports will look like in the future,Data Science,blog
1738,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2020-03-26T21:39:16.764Z,What sports will look like in the future,Information Technology,blog
1739,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2020-03-26T21:39:16.764Z,What sports will look like in the future,Artificial Intelligence,blog


In [0]:
main_df.describe()

,user_id,post_id,timestamp,title,category,post_type
count,1750,1750,1750,1750,1750,1750
unique,88,231,626,227,234,3
top,5d60098a653a331687083238,5ec7abfdec493f4a26558860,2020-05-26T12:33:48.981Z,FASHION ILLUSTRATION (OP ART),Illustration,blog
freq,261,63,7,63,78,846


**Now the merging is done and we have our actuall dataframe to work on now lets move to the nect step which is filtering using Collaborative Filtering **

# **Collaborative Filtering**

**So we are using user-user collaberative filtering. And we are using cosine similarity to determine the similarity between users. For that we need to build a matrix. categories will be the columns and user_ids will be the rows. And we are using K near neighbour model to determine the similar users.**

## CONSTRUCTING THE MATRIX

**We will create a sparse matrix of mXn. All the unique categories will be the column values and user_ids will be the row value. And the cell values will be the number times a user has viwed a category. The higher the cell value the higher the users interest for that particular category**

**Lets find all the unique user_ids and categories**

In [0]:
users = list(main_df['user_id'].unique())
len(users)

'5df49b32cc709107827fb3c7'

In [0]:
categories = list(main_df['category'].unique())
len(categories)

234

**Our matrix size will be of size 88X234**

In [0]:
user_mat = [[] for i in range(len(users))]
for i in range(len(users)):
    for j in range(len(categories)):
        value = len(main_df[(main_df['user_id']==users[i])&(main_df['category']==categories[j])])
        user_mat[i].append(value)

In [0]:
for i in user_mat[0]:
    print(i, end=" ")

4 3 3 3 2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 0 0 1 2 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

**Now we will convert the list to a actual sparse matrix using scipy.sparse library**

In [0]:
from scipy.sparse import csr_matrix # for creating a sparse matrix
user_mat = csr_matrix(user_mat)

In [0]:
user_mat

<88x234 sparse matrix of type '<class 'numpy.longlong'>'
	with 1034 stored elements in Compressed Sparse Row format>

**As we have made our matrix lets train the model.**

## TRAINING THE MODEL

**So we are using K nearest neighbour model. And the similarity constraint we are using is cosine similarity.**

**Importing the model**

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=15)

In [0]:
model_knn.fit(user_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                 radius=1.0)

**We have trained our model. Now lets make a function which will take users_id as input and recommend categories that the user would like**

In [0]:
def recommender(user_id, data=user_mat, model=model_knn):
    model.fit(data)
    index = users.index(user_id)
    current_user = main_df[main_df['user_id']==user_id]
    distances, indices = model.kneighbors(data[index], 15)
    recomendation = []
    for i in indices[0]:
        user = main_df[main_df['user_id']==users[i]]
        for i in user['category'].unique():
            if i not in current_user['category'].unique():
                recomendation.append(i)
    print(recomendation)
    print(indices)

**What we did above there is first we found out 15 similar users for our current user and we recomended those posts to the user which he/she hasnot viewed yet but similar users have.Now lets test our function for a user**

**Lets see what the system recomendation for the first user before that lets see the users categories that he has already viewed**

In [0]:
main_df[main_df['user_id']==users[0]]['category'].unique()

array(['Visual Arts', 'Graphic Design', 'Artistic design', 'Graphic',
       'Illustration', 'Photography', 'Architecture', 'Logo Design',
       'Art; Science', 'Science;Technology', 'Fashion Desigining',
       'Pattern & Culture', 'Fashion Textile', 'Fashion Trends',
       'Fashion Illustration', 'Pattern Cutting', 'Fashion Communication',
       'Art', 'Calligraphy'], dtype=object)

**These are the categories the user has viewed now lets see the recomendations for our user**

In [0]:
recommender(users[0])

['Fashion Portfoilio', 'Fashion Design', 'Watercolours', 'Drawings', 'Sketch Video', 'Political Science', 'Colonialism In India', 'Legal Studies', 'Labor Law', 'Banking', 'Banking Companies', 'Professionalism', 'Fashion Portfoilio', 'Painting', 'Craft', 'Technology', 'Drawings', 'Business', 'Learning', 'Literature', 'Stories', 'Fictions', 'Movements in Literature', 'Fashion Portfoilio', 'Business Strategies', 'Business Enviorment', 'New Venture Planning', 'Foreign Business', 'Business Organisation', 'Fashion Design', 'Watercolours', 'Conceptual', 'Pen and ink', 'Mixed Media', '2D Composition', 'Communication', 'Basics of Communiaction', 'Sketch Video', 'Professionalism', 'Biotechnology', 'Molecular Biology', 'Fine Arts', 'Painting', 'Management', 'Business Management', 'Technology Mangement', 'Managerial Activity', 'Creative And Lateral Management', 'Economics', 'Economic Policies', 'Break even Point', 'Foreign Economy', 'Computer Technology', 'Operating Systems', 'Fashion Manufacturin

**Looks like the system is suggesting posts of users interest and also some new categories for the user.**
**The List below suggests the similar users to the current user.**

**So This ends our Collaborative Filtering. Lets move on to Content based filtering**

# CONTENT BASED FILTERING